### 코드 작성일 : 24.7.29 
### 코드 목표 : 대표 메뉴 추출 

In [1]:
import os
#from hanspell import spell_checker

if 'original_dir' not in globals() :
    original_dir = os.getcwd()
    original_dir = os.path.dirname(os.path.dirname(os.path.dirname(os.path.dirname(os.path.dirname(original_dir)))))
    os.chdir(original_dir)
exec(open('setup/default.py').read())

/Users/al01989093/anaconda3/envs/first_pytorch/lib/python3.11/site-packages/fuzzywuzzy/fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


In [2]:
menu_w_sold_out = pd.read_csv(os.getcwd()+'/data/preproc/main/menu_w_soldout.csv')

#### 대표 메뉴 추출 
##### 클렌징 

In [3]:
# 클렌징 
def filter_text(text):
    # '>'가 있는 경우 '>' 뒤의 텍스트 추출
    if '>' in text:
        text = text.replace('>,', '>') # 만약 <~>,이런식으로 붙어있으면 변경해주기
        text = text.split('>')[-1]
    # ','가 있는 경우 ',' 앞의 텍스트 추출
    if ',' in text:
        text = text.split(',')[0]
    # 특수문자 이후의 텍스트 제거
    text = re.sub(r'[(*&].*$', '', text).strip()
    return text

menu_w_sold_out['first_menu_unpreproc'] = menu_w_sold_out['course'].apply(filter_text)


##### 맞춤법 교정 

In [4]:
import string

# 특수 기호와 숫자를 제거하는 함수 + 함박 뒤에 스테이크 추가 
def replace_text(text, replacements):
    # 교체 작업
    for key, value in replacements.items():
        text = text.replace(key, value)
    
    # 숫자와 특수 기호 제거
    text = ''.join(c for c in text if c not in string.punctuation and not c.isdigit())
    
    # "함박" 뒤에 "스테이크" 추가
    if "함박" in text and "스테이크" not in text:
        text = text.replace("함박", "함박 스테이크")
    elif "함바그" in text and "스테이크" not in text:
        text = text.replace("함바그", "함박 스테이크")

    if text.endswith("카레") or text.endswith("커리"):
        text += " 라이스"
    return text


# Replacements dictionary
replacements = {
    "돈가스": "돈까스",
    "치킨가스": "치킨까스",
    "자장": "짜장",
    "코든블루": "코돈브루",
    "코든브루": "코돈브루",
    "코돈블루": "코돈브루",
    "플래이트": "",
    "플레이트": "",
    "플래터": "",
    "플레터": "",
    "정식": "",
    "뼈없는" : "",
    "소세지": "소시지",
    "돈유": "돈육",
    "함바그" : "함박",
    "커틀렛" : "커틀릿",
    "오무" : "오므",
    "뽀모로" : "뽀모도로",
    "복이" : "볶이",
    "블백" : "불백",
    "런치" : "", 
    "스페셜" : "",
    "스램":"스팸",
    "므쌉":"무쌉",
    "클럼블" : "크럼블",
    "카케":"카레"
}

# DataFrame에 함수 적용
menu_w_sold_out['first_menu'] = menu_w_sold_out['first_menu_unpreproc'].apply(lambda x: replace_text(x, replacements))


In [5]:
# 성능이 그렇게 좋지 않아서 안하는게 나아보임. 
# def spell_check(text):
#     result = spell_checker.check(text).checked
#     return result

# # 진행 상황을 표시하면서 apply 적용
# menu_w_sold_out['first_menu'] = menu_w_sold_out['first_menu_unpreproc'].progress_apply(spell_check)

In [6]:
menu_w_sold_out.to_csv(original_dir+'/data/preproc/main/feature_lab_v1/menu_w_soldout_w_first_menu.csv',index =False)

In [7]:
pd.DataFrame(data={'menu':menu_w_sold_out['first_menu'].unique().tolist()}).to_csv(original_dir+'/data/preproc/main/feature_lab_v1/unique_first_menu.csv',index =False)